# uniswap

> Get Raw Uniswap swap data

In [2]:
#| default_exp uniswap

In [2]:
# | export
import cryo
import jupyter_black
import os
import polars as pl
import time

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

jupyter_black.load()

In [ ]:
cd ../

In [32]:
# | export
def create_data_filepaths():
    """
    Creates folders for storing raw and combined data if they do not already exist.
    This function checks for the presence of 'data/raw' and 'data/combined' directories.
    If these directories do not exist, they are created. If they already exist,
    a message indicating their existence is printed.
    """
    raw_data_path = "data/raw"
    if not os.path.exists(raw_data_path):
        os.makedirs(raw_data_path)
        print("Data folder created.")
    else:
        print("Data folder already exists.")

    combined_data_path = "data/combined"
    if not os.path.exists(combined_data_path):
        os.makedirs(combined_data_path)
        print("Data folder created.")
    else:
        print("Data folder already exists.")

# | export
def get_univ2_swaps(start_block: int = 18039828, end_block: int = 18473546, n_error_threshold: int=1, retry_threshold: int=5, ):
    """
    Fetches swap data from Uniswap V2 between specified block ranges.
    The function queries and retrieves swap data, attempting retries up to 'retry_threshold'
    if errors exceed 'n_error_threshold'. The retrieved data is saved as a Parquet file.

    :param start_block: The starting block number for the query (inclusive).
    :param end_block: The ending block number for the query (inclusive).
    :param n_error_threshold: The number of allowed errors before retrying the query.
    :param retry_threshold: The maximum number of retries for the query.
    """
    create_data_filepaths()
    n_errored = n_error_threshold + 1
    retry_count = 0

    # make cryo query
    while retry_count < retry_threshold:
        print(f'Retry count: {retry_count}')
        retry_count += 1
        if n_error_threshold < n_errored:

            output: dict[str] = cryo.freeze(
            "logs",
            # label="uniswap_v2_swaps",
            blocks=[f"{start_block}:{end_block}"], 
            rpc="https://eth.merkle.io",
            reorg_buffer=1000,
            max_concurrent_chunks=15, 
            chunk_size=1000,
            inner_request_size=200,
            output_dir="data/raw",
            subdirs=["univ2_swaps"],
            event_signature="Swap(address indexed sender, uint amount0In, uint amount1In, uint amount0Out, uint amount1Out, address indexed to)",
            topic0=["0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822"],
            hex=True
            )
                
            n_errored = output["n_errored"]
            print(f'Number of errors: {n_errored}')
        if n_errored == 0:
            print(f'{n_errored} == 0. Done!')
            break
    
    # save to single file
    univ2_swaps = pl.scan_parquet('data/raw/univ2_swaps/*.parquet').collect(streaming=True)
    univ2_swaps.write_parquet('data/combined/univ2_swaps.parquet')

# | export
def get_univ2_pools(n_error_threshold: int=1, retry_threshold: int=5):
    """
    Queries and retrieves Uniswap V2 pool data, with retry logic.
    The function attempts to fetch data, with the number of retries defined by 'retry_threshold',
    and stops if errors are less than or equal to 'n_error_threshold'. The result is stored as a Parquet file.

    :param n_error_threshold: The number of allowed errors before retrying the query.
    :param retry_threshold: The maximum number of retries for the query.
    """
    create_data_filepaths()
    n_errored = n_error_threshold + 1
    retry_count = 0

    # make cryo query
    while retry_count < retry_threshold:
        print(f'Retry count: {retry_count}')
        retry_count += 1
        if n_error_threshold < n_errored:
            output: dict[str] = cryo.freeze(
                "logs",
                blocks=["10M:"], 
                rpc="https://eth.merkle.io",
                reorg_buffer=1000,
                max_concurrent_chunks=15, 
                chunk_size=100000,
                inner_request_size=1000,
                output_dir="data/raw",
                subdirs=["univ2_pools"],
                contract=["0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f"],
                event_signature="PairCreated(address indexed token0, address indexed token1, address pair, uint)",
                topic0=["0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9"],
                hex=True,
            )
            n_errored = output["n_errored"]
            print(f'Number of errors: {n_errored}')
        if n_errored == 0:
            print(f'{n_errored} == 0. Done!')
            break
    
    # save to single file
    univ2_pools = pl.scan_parquet('data/raw/univ2_pools/*.parquet').collect(streaming=True)
    univ2_pools.write_parquet('data/combined/univ2_pools.parquet')

# | export
def get_erc20_metadata(tokens: list[str], n_error_threshold: int=1, retry_threshold: int=5):
    """
    Retrieves metadata for a list of ERC20 tokens.
    The function performs a query for each token in the 'tokens' list, with retry logic
    based on 'n_error_threshold' and 'retry_threshold'. The collected data is then saved as a Parquet file.

    :param tokens: A list of ERC20 token addresses to query.
    :param n_error_threshold: The number of allowed errors before retrying the query.
    :param retry_threshold: The maximum number of retries for the query.
    """
    create_data_filepaths()
    n_errored = n_error_threshold + 1
    retry_count = 0

    while retry_count < retry_threshold:
        print(f'Retry count: {retry_count}')
        retry_count += 1
        if n_errored > n_error_threshold:
            output: dict[str] = cryo.freeze(
            "erc20_metadata",
            blocks=["18.78M"], 
            rpc="https://eth.merkle.io",
            chunk_size=7200,
            inner_request_size=1000,
            output_dir="data/raw",
            subdirs=["univ2_tokens"],
            contract=tokens,
            hex=True,
        )
            n_errored = output["n_errored"]
            print(f'Number of errors: {n_errored}')
        if n_errored == 0:
            print(f'{n_errored} == 0. Done!')
            break
    
    # save to single file
    univ2_tokens = pl.scan_parquet('data/raw/univ2_tokens/*.parquet').collect(streaming=True)
    univ2_tokens.write_parquet('data/combined/univ2_tokens.parquet')

In [11]:
start_time = time.time()
# First get the swaps for 15,000 blocks. 
get_univ2_swaps(start_block=18039828, end_block=18039828+15000)
end_time = time.time()

print(f'Elapsed time: {end_time - start_time} seconds')

Data folder created.
Data folder created.
Retry count: 0
cryo parameters
───────────────
- version: 0.3.0-3-g041f38b
- data: 
    - datatypes: logs
    - blocks: n=15,000 min=18,039,828 max=18,054,827 align=no reorg_buffer=1000
    - topic0s: n=15 min=0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822 max=0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822
- source: 
    - network: ethereum
    - rpc url: https://eth.merkle.io
    - max requests per second: unlimited
    - max concurrent requests: unlimited
    - max concurrent chunks: 15
    - inner request size: 200
- output: 
    - chunk size: 1,000
    - chunks to collect: 15 / 15
    - output format: parquet
    - output dir: /home/evan/Documents/cryo_uniswap/data/raw
    - report file: $OUTPUT_DIR/.cryo/reports/2023-12-23_16-17-37.424483.json


schema for logs
───────────────
- block_number: uint32
- transaction_index: uint32
- log_index: uint32
- transaction_hash: hex
- address: hex
- topic0: hex
-

In [47]:
# read the data, select column float values, consolidate swap amount values, and rename columns.
univ2_swaps_df = (pl.read_parquet('data/combined/univ2_swaps.parquet').select(
    'block_number', 'transaction_hash', 'address', 'event__to',
    'event__amount0In_f64', 'event__amount1In_f64',
    'event__amount0Out_f64', 'event__amount1Out_f64',
    'event__sender', 
    ).with_columns([
    pl.when(pl.col("event__amount0In_f64") > 0).then(pl.col("event__amount0In_f64")).otherwise(pl.col("event__amount0Out_f64") * -1).alias("swaps_amount0"),
    pl.when(pl.col("event__amount1In_f64") > 0).then(pl.col("event__amount1In_f64")).otherwise(pl.col("event__amount1Out_f64") * -1).alias("swaps_amount1"),
]).rename({
    'transaction_hash': 'swaps_transaction_id', 
    'address': 'swaps_pool_id', 
    'event__to': 'swaps_recipient', 
    'event__sender': 'swaps_sender', 
    # 'event__token0': 'swaps_token0_address', 
    # 'event__token1': 'swaps_token1_address', 
}).drop('event__amount0In_f64', 'event__amount0Out_f64', 'event__amount1In_f64', 'event__amount1Out_f64'))

In [48]:
univ2_swaps_df.head(5)

block_number,swaps_transaction_id,swaps_pool_id,swaps_recipient,swaps_sender,swaps_amount0,swaps_amount1
u32,str,str,str,str,f64,f64
18039828,"""0xe6698c27a7cf482fb233a00bc567b61d23bd125780e157ebb873d13b88c0b52f""","""0x7626367d028b969a7930c28ee3ae18b20e83ecda""","""0x69077669f0875064ec1323fbe91462ac7dd9ec80""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",-204094854136496470000000,500000000000000000
18039828,"""0x3487a6e9d4ae132a63e233df33fcbaba09980e03691b443316231d7a9e8ccaf0""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",-745901651918848,286823911801225200
18039828,"""0xaccd5c41fdc3dcc7adb7ab781504c9595bbc7dba4047dd87a4574ea5f89cff29""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0xf375be113180dde790314a20e55b93b63b6f0f5e""","""0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad""",-1217107582095639,500000000000000000
18039828,"""0xb0deab4818f94ca1a321a8de44225fdffd6ea38157ea0c7311ca155f7a26cd46""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",731058192934503,-303506630232768500
18039828,"""0x44cd392f3ba7047987688b5fc932727c6b6148d1dfb398662347591294e74daf""","""0xe3550f8399c8622ab2c0b2e48856598321cb9b56""","""0xabf864065fdeb8996eb5469d6c25aae84507d231""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",50292615743099710000000000,-2546988193917007000


In [31]:
# # TODO 12/23/23 - Is it possible to speed up the list of pool contract addresses? Currently `get_univ2_pools()` queries logs for pool creation events and then returns pool information.
# unique_pools = swaps_sample['address'].unique().to_list()

# unique_pools[:2]

# print(len(unique_pools))

# # ! This needs to be changed to work with function calls
# output: dict[str] = cryo.freeze(
#     "logs",
#     blocks=["18.78M:"], 
#     rpc="https://eth.merkle.io",
#     reorg_buffer=1000,
#     max_concurrent_chunks=15, 
#     chunk_size=100000,
#     inner_request_size=1000,
#     output_dir="data/raw",
#     subdirs=["univ2_pool_single"],
#     contract=['0x475d77a57b4ede703521c711474fc386a98cc0de'],
#     event_signature="PairCreated(address indexed token0, address indexed token1, address pair, uint)",
#     topic0=["0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9"],
#     hex=True,
# )

In [33]:
start_time = time.time()
# First get the swaps for 15,000 blocks. 
get_univ2_pools()
end_time = time.time()

print(f'Elapsed time: {end_time - start_time} seconds')

Data folder already exists.
Data folder already exists.
Retry count: 0
cryo parameters
───────────────
- version: 0.3.0-3-g041f38b
- data: 
    - datatypes: logs
    - blocks: n=8,800,000 min=10,000,000 max=18,799,999 align=no reorg_buffer=1000
    - addresses: n=88 min=0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f max=0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f
    - topic0s: n=88 min=0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9 max=0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9
- source: 
    - network: ethereum
    - rpc url: https://eth.merkle.io
    - max requests per second: unlimited
    - max concurrent requests: unlimited
    - max concurrent chunks: 15
    - inner request size: 1000
- output: 
    - chunk size: 100,000
    - chunks to collect: 88 / 88
    - output format: parquet
    - output dir: /home/evan/Documents/cryo_uniswap/data/raw
    - report file: $OUTPUT_DIR/.cryo/reports/2023-12-23_17-02-04.022668.json


schema for logs
─────

In [34]:
# load univ2 pool data and get tokens list
univ2_pools_raw = pl.read_parquet('data/combined/univ2_pools.parquet')
univ2_pools_tokens = list(set(univ2_pools_raw['event__token1'].to_list() + univ2_pools_raw['event__token0'].to_list()))


start_time = time.time()
# First get the swaps for 15,000 blocks. 
get_erc20_metadata(univ2_pools_tokens)
end_time = time.time()

print(f'Elapsed time: {end_time - start_time} seconds')

Data folder already exists.
Data folder already exists.
Retry count: 0
cryo parameters
───────────────
- version: 0.3.0-3-g041f38b
- data: 
    - datatypes: erc20_metadata
    - block: 18,780,000
    - addresses: n=286,705 min=0x00000000000045166c45af0fc6e4cf31d9e14b9a max=0xfffffffff15abf397da76f1dcc1a1604f45126db
- source: 
    - network: ethereum
    - rpc url: https://eth.merkle.io
    - max requests per second: unlimited
    - max concurrent requests: unlimited
    - max concurrent chunks: 4
- output: 
    - chunk size: 1
    - chunks to collect: 1 / 1
    - output format: parquet
    - output dir: /home/evan/Documents/cryo_uniswap/data/raw
    - report file: $OUTPUT_DIR/.cryo/reports/2023-12-23_17-03-01.800687.json


schema for erc20_metadata
─────────────────────────
- block_number: uint32
- erc20: hex
- name: string
- symbol: string
- decimals: uint32
- chain_id: uint64

sorting erc20_metadata by: symbol, block_number

other available columns: [none]


collecting data
─────────

In [40]:
univ2_tokens_df = pl.read_parquet('data/combined/univ2_tokens.parquet')

In [43]:
# this is the dataset we join to
univ2_pools_df= pl.read_parquet('data/combined/univ2_pools.parquet')

In [45]:
univ2_pools_df.head(5)

block_number,transaction_index,log_index,transaction_hash,address,topic0,chain_id,event__token0,event__token1,event__pair,event___binary,event___string,event___f64
u32,u32,u32,str,str,str,u64,str,str,str,str,str,f64
10008355,38,34,"""0xd07cbde817318492092cc7a27b3064a69bd893c01cb593d6029683ffd290ab3a""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9""",1,"""0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc""","""0x0000000000000000000000000000000000000000000000000000000000000001""","""1""",1
10008500,35,28,"""0xb0621ca74cee9f540dda6d575f6a7b876133b42684c1259aaeb59c831410ccb2""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9""",1,"""0x8e870d67f660d95d5be530380d0ec0bd388289e1""","""0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48""","""0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee""","""0x0000000000000000000000000000000000000000000000000000000000000002""","""2""",2
10014178,38,27,"""0x98f92a0b6063b0b581722cecf8ef01131c24c4377e6d3323d9184d1a262fc168""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9""",1,"""0x06af07097c9eeb7fd685c692751d5c66db49c215""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0x12ede161c702d1494612d19f05992f43aa6a26fb""","""0x0000000000000000000000000000000000000000000000000000000000000003""","""3""",3
10042267,70,56,"""0xc4c1840db940f5075c5404266efe50d7c65bc079f318ab8e5a10b8a432b49d30""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9""",1,"""0x6b175474e89094c44da98b954eedeac495271d0f""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""0xa478c2975ab1ea89e8196811f51a7b7ade33eb11""","""0x0000000000000000000000000000000000000000000000000000000000000004""","""4""",4
10060679,59,59,"""0xee29b2a9306dab868b53919836c4a81feca435de6902c230c291b576ec9fa680""","""0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f""","""0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9""",1,"""0x408e41876cccdc0f92210600ef50372656052a38""","""0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48""","""0x07f068ca326a469fc1d87d85d448990c8cba7df9""","""0x0000000000000000000000000000000000000000000000000000000000000005""","""5""",5


In [49]:
univ2_swaps_df.head(5)

block_number,swaps_transaction_id,swaps_pool_id,swaps_recipient,swaps_sender,swaps_amount0,swaps_amount1
u32,str,str,str,str,f64,f64
18039828,"""0xe6698c27a7cf482fb233a00bc567b61d23bd125780e157ebb873d13b88c0b52f""","""0x7626367d028b969a7930c28ee3ae18b20e83ecda""","""0x69077669f0875064ec1323fbe91462ac7dd9ec80""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",-204094854136496470000000,500000000000000000
18039828,"""0x3487a6e9d4ae132a63e233df33fcbaba09980e03691b443316231d7a9e8ccaf0""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",-745901651918848,286823911801225200
18039828,"""0xaccd5c41fdc3dcc7adb7ab781504c9595bbc7dba4047dd87a4574ea5f89cff29""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0xf375be113180dde790314a20e55b93b63b6f0f5e""","""0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad""",-1217107582095639,500000000000000000
18039828,"""0xb0deab4818f94ca1a321a8de44225fdffd6ea38157ea0c7311ca155f7a26cd46""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",731058192934503,-303506630232768500
18039828,"""0x44cd392f3ba7047987688b5fc932727c6b6148d1dfb398662347591294e74daf""","""0xe3550f8399c8622ab2c0b2e48856598321cb9b56""","""0xabf864065fdeb8996eb5469d6c25aae84507d231""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",50292615743099710000000000,-2546988193917007000


In [62]:
univ2_swaps_final = (
    univ2_swaps_df
    .join(univ2_pools_df.select('event__pair', 'event__token0', 'event__token1').rename({'event__pair': 'swaps_pool_id'}), on='swaps_pool_id', how='left')
    .join(univ2_tokens_df.select('erc20', 'name', 'symbol', 'decimals').rename({'erc20': 'event__token0'}), on='event__token0', how='left', suffix='_token0')
    .join(univ2_tokens_df.select('erc20', 'name', 'symbol', 'decimals').rename({'erc20': 'event__token1'}), on='event__token1', how='left', suffix='_token1')
    )

In [63]:
univ2_swaps_final.head(5)

block_number,swaps_transaction_id,swaps_pool_id,swaps_recipient,swaps_sender,swaps_amount0,swaps_amount1,event__token0,event__token1,name,symbol,decimals,name_token1,symbol_token1,decimals_token1
u32,str,str,str,str,f64,f64,str,str,str,str,u32,str,str,u32
18039828,"""0xe6698c27a7cf482fb233a00bc567b61d23bd125780e157ebb873d13b88c0b52f""","""0x7626367d028b969a7930c28ee3ae18b20e83ecda""","""0x69077669f0875064ec1323fbe91462ac7dd9ec80""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",-204094854136496470000000,500000000000000000,"""0x667210a731447f8b385e068205759be2311b86d4""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""ETFTheToken""","""ETF""",18,"""WrappedEther""","""WETH""",18
18039828,"""0x3487a6e9d4ae132a63e233df33fcbaba09980e03691b443316231d7a9e8ccaf0""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",-745901651918848,286823911801225200,"""0x9297dc566b901a75640666fd44c642e53d8e7de1""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""HarryPotterVsPepe""","""HEPE""",9,"""WrappedEther""","""WETH""",18
18039828,"""0xaccd5c41fdc3dcc7adb7ab781504c9595bbc7dba4047dd87a4574ea5f89cff29""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0xf375be113180dde790314a20e55b93b63b6f0f5e""","""0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad""",-1217107582095639,500000000000000000,"""0x9297dc566b901a75640666fd44c642e53d8e7de1""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""HarryPotterVsPepe""","""HEPE""",9,"""WrappedEther""","""WETH""",18
18039828,"""0xb0deab4818f94ca1a321a8de44225fdffd6ea38157ea0c7311ca155f7a26cd46""","""0xacdb4427d404d9977b74d19fcd075eb9b43ff274""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""","""0x00000000a991c429ee2ec6df19d40fe0c80088b8""",731058192934503,-303506630232768500,"""0x9297dc566b901a75640666fd44c642e53d8e7de1""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""HarryPotterVsPepe""","""HEPE""",9,"""WrappedEther""","""WETH""",18
18039828,"""0x44cd392f3ba7047987688b5fc932727c6b6148d1dfb398662347591294e74daf""","""0xe3550f8399c8622ab2c0b2e48856598321cb9b56""","""0xabf864065fdeb8996eb5469d6c25aae84507d231""","""0x7a250d5630b4cf539739df2c5dacb4c659f2488d""",50292615743099710000000000,-2546988193917007000,"""0x9138c8779a0ac8a84d69617d5715bd8afa23c650""","""0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2""","""FloorCheeseBurger""","""FLRBRG""",18,"""WrappedEther""","""WETH""",18
